## LSTMs

## Demo 1 - torch.nn.LSTM

No PytTorch, a classe [torch.nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) provê a implementação de uma célula LSTM. Essa classe implementa as seguintes equações:

![texto do link](https://i.imgur.com/A9reTrh.png)

Da documentação do PyTorch:

- `input_size` – The number of expected features in the input x
- `hidden_size` – The number of features in the hidden state h
- `num_layers` – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two LSTMs together to form a stacked LSTM, with the second LSTM taking in outputs of the first LSTM and computing the final results. Default: 1

Como exemplo, considere uma arquitetura LSTM em que foram definidos:
- $c_𝑡 \in \mathbb{R}^2$
- $ℎ_𝑡 \in \mathbb{R}^2$
- $𝑥_𝑡 \in \mathbb{R}^3$

![texto do link](https://i.imgur.com/Ifn6yVs.png)

Cada camada sigmoid, tanh ou de estado oculto em uma célula LSTM é na verdade uma rede neural feed forward de uma única camada oculta, cujo número de neurônios é definido pelo parâmetro `hidden_size`. 

- Se definirmos `hidden_size = 2`, cada célula LSTM terá redes neurais com 2 neurônios em sua camada oculta.

- Se definirmos `input_size = 3`, cada elemento da sequência de entrada será representado por um vetor de três dimensões.

In [17]:
import torch.nn as nn
hidden_size = 2
input_size = 3
lstm1 = nn.LSTM(input_size, hidden_size, num_layers=1)

Uma célula LSTM possui as seguintes propriedades (texto a seguir também retirado da [documentação do PyTorch](https://pytorch.org/docs/stable/nn.html)):

- weight_ih_l[k] – the learnable input-hidden weights of the $\text{k}^{th}$ ($W_{ii}|W_{if}|W_{ig}|W_{io}$), of shape (4$\times$hidden_size, input_size) for $k = 0$.

- weight_hh_l[k] – the learnable hidden-hidden weights of the $\text{k}^{th}k$ layer ($W_{hi}|W_{hf}|W_{hg}|W_{ho}$), of shape (4*hidden_size, hidden_size)


Para a célula LSTM apresentada nesta seção, valem as seguintes dimensões:

$$
W_{ii}, W_{if}, W_{ig}, W_{io} \in \mathbb{R}^{2 \times 3}
$$

In [18]:
print(lstm1.weight_ih_l0.size())
print(lstm1.weight_ih_l0)

torch.Size([8, 3])
Parameter containing:
tensor([[-0.1932, -0.5081,  0.0987],
        [-0.0793,  0.5171, -0.2966],
        [-0.1412,  0.5200, -0.3319],
        [ 0.4398, -0.1529, -0.0209],
        [ 0.5111, -0.3910,  0.1674],
        [ 0.0869,  0.4588,  0.3659],
        [-0.5566, -0.5922, -0.2575],
        [-0.4459,  0.6885, -0.0592]], requires_grad=True)


Para a célula LSTM apresentada nesta seção, valem as seguintes dimensões:

$$
W_{hi}, W_{hf}, W_{hg}, W_{ho} \in \mathbb{R}^{2 \times 2}
$$

In [20]:
print(lstm1.weight_hh_l0.size())
print(lstm1.weight_hh_l0)

torch.Size([8, 2])
Parameter containing:
tensor([[-0.6337,  0.1353],
        [ 0.3896, -0.4928],
        [-0.2574,  0.5458],
        [ 0.4612,  0.3946],
        [-0.4717, -0.0333],
        [-0.4188, -0.2073],
        [-0.3588,  0.6900],
        [ 0.6157, -0.0471]], requires_grad=True)


## Demo 2 - Previsão de séries multivariadas

**Créditos**: o exemplo desta seção foi adaptado do encontrado em [Multivariate input LSTM in pytorch](https://stackoverflow.com/questions/56858924/multivariate-input-lstm-in-pytorch).

Esta seção apresenta um exemplo muito simples de treinamento de um modelo de rede LSTM para predição no contexto de uma série temporal multivariada.

> Considere uma série temporal multivariada na qual há três observações (variáveis medidas) por passo de tempo. Dessas três variáveis, uma dela é alvo (i.e., a que desejamos predizer). As outras duas serão usadas como variáveis independentes. Considere também a tarefa de ajustar um modelo de predição no qual são usadas observações de três passos de tempo no passado para predizer o valor da variável de interesse no passo de tempo atual.

### Preparação dos dados

Vamos de início gerar um conjunto de dados sintético. Esse conjunto de dados simula uma série temporal multivariada na qual, a cada passo de tempo são observadas (medidas) três variáveis.

In [8]:
import random
import numpy as np
import torch

# multivariate data preparation
from numpy import array
from numpy import hstack
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
 
# define input sequence
in_seq1 = array([x for x in range(0,100,10)])
in_seq2 = array([x for x in range(5,105,10)])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

A série temporal multivariada gerada pelo código acima é apresentada a seguir. Repare que há 10 passos de tempo, em cada um dos quais, três observações são realizadas.

In [9]:
dataset

array([[  0,   5,   5],
       [ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [10]:
n_timesteps = 3 # this is number of timesteps

In [11]:
# convert dataset into input/output
X, y = split_sequences(dataset, n_timesteps)
print(X.shape, y.shape)

(8, 3, 2) (8,)


In [12]:
print(X)

[[[ 0  5]
  [10 15]
  [20 25]]

 [[10 15]
  [20 25]
  [30 35]]

 [[20 25]
  [30 35]
  [40 45]]

 [[30 35]
  [40 45]
  [50 55]]

 [[40 45]
  [50 55]
  [60 65]]

 [[50 55]
  [60 65]
  [70 75]]

 [[60 65]
  [70 75]
  [80 85]]

 [[70 75]
  [80 85]
  [90 95]]]


In [13]:
print(y)

[ 45  65  85 105 125 145 165 185]


### Construção do modelo

Esta seção apresenta a implemetação do modelo em PyTorch. Como de praxe, a classe correspondente é derivada de `torch.nn.Module`.

In [14]:
class MV_LSTM(torch.nn.Module):
    def __init__(self,n_features,seq_length):
        super(MV_LSTM, self).__init__()
        self.n_features = n_features
        self.seq_len = seq_length
        self.n_hidden = 20 # number of hidden states
        self.n_layers = 1 # number of LSTM layers (stacked)
    
        self.l_lstm = torch.nn.LSTM(input_size = n_features, 
                                 hidden_size = self.n_hidden,
                                 num_layers = self.n_layers, 
                                 batch_first = True)
        # according to pytorch docs LSTM output is 
        # (batch_size,seq_len, num_directions * hidden_size)
        # when considering batch_first = True
        self.l_linear = torch.nn.Linear(self.n_hidden*self.seq_len, 1)
        
    
    def init_hidden(self, batch_size):
        # even with batch_first = True this remains same as docs
        hidden_state = torch.zeros(self.n_layers,batch_size,self.n_hidden)
        cell_state = torch.zeros(self.n_layers,batch_size,self.n_hidden)
        self.hidden = (hidden_state, cell_state)
    
    
    def forward(self, x):        
        batch_size, seq_len, _ = x.size()
        
        lstm_out, self.hidden = self.l_lstm(x,self.hidden)
        # lstm_out(with batch_first = True) is 
        # (batch_size,seq_len,num_directions * hidden_size)
        # for following linear layer we want to keep batch_size dimension and merge rest       
        # .contiguous() -> solves tensor compatibility error
        x = lstm_out.contiguous().view(batch_size,-1)
        return self.l_linear(x)

### Configuração

In [15]:
n_features = 2 # this is number of parallel inputs

# create NN
mv_net = MV_LSTM(n_features, n_timesteps)
criterion = torch.nn.MSELoss() # reduction='sum' created huge loss value
optimizer = torch.optim.Adam(mv_net.parameters(), lr=1e-1)

train_episodes = 500
batch_size = 16

### Treinamento

In [16]:
mv_net.train()
for t in range(train_episodes):
    for b in range(0,len(X),batch_size):
        inpt = X[b:b+batch_size,:,:]
        target = y[b:b+batch_size]    
        
        x_batch = torch.tensor(inpt,dtype=torch.float32)    
        y_batch = torch.tensor(target,dtype=torch.float32)
    
        mv_net.init_hidden(x_batch.size(0))

        output = mv_net(x_batch) 
        loss = criterion(output.view(-1), y_batch)  
        
        loss.backward()
        optimizer.step()        
        optimizer.zero_grad() 
    print('step : ' , t , 'loss : ' , loss.item())

step :  0 loss :  15388.94921875
step :  1 loss :  15017.09765625
step :  2 loss :  14673.6796875
step :  3 loss :  14356.7880859375
step :  4 loss :  14033.16796875
step :  5 loss :  13554.828125
step :  6 loss :  12812.4365234375
step :  7 loss :  12329.8701171875
step :  8 loss :  11862.169921875
step :  9 loss :  11422.3623046875
step :  10 loss :  10983.6669921875
step :  11 loss :  10548.66015625
step :  12 loss :  10119.884765625
step :  13 loss :  9699.3212890625
step :  14 loss :  9288.4794921875
step :  15 loss :  8887.8359375
step :  16 loss :  8496.3037109375
step :  17 loss :  8102.515625
step :  18 loss :  7546.8466796875
step :  19 loss :  7158.310546875
step :  20 loss :  6808.65234375
step :  21 loss :  6470.984375
step :  22 loss :  6146.25634765625
step :  23 loss :  5833.763671875
step :  24 loss :  5506.349609375
step :  25 loss :  5056.294921875
step :  26 loss :  4779.8896484375
step :  27 loss :  4517.98779296875
step :  28 loss :  4270.06640625
step :  29 loss 